<a href="https://colab.research.google.com/github/ketanhdoshi/ml/blob/master/examples/Tensorboard_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set the LOGDIR correctly to use Tensorboard
LOG_DIR = 'tbtry'

### Set up Tensorboard

**To view Tensorboard output locally, use ngrok to tunnel traffic to localhost. First, download and unzip ngrok on the Colab server**

In [ ]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

**Get TensorBoard running in the background**

In [ ]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

**Launch ngrok background process**

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

**We get the public URL where we can access the colab TensorBoard web page. This will output a URL you can click on**

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://58f1c1f8.ngrok.io


### Pytorch

In [ ]:
import numpy as np
from pathlib import *
import torch
print('Pytorch version:', torch.__version__)
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

#----------------------------------------------------
# Histogram
#----------------------------------------------------
def add_hist(writer):
  for i in range(10):
    x = np.random.random(1000)
    writer.add_histogram('distribution centers', x + i, i)

#----------------------------------------------------
# Mesh
#----------------------------------------------------
def add_mesh(writer):
  vertices_tensor = torch.as_tensor([[1, 1, 1],
                                     [-1, -1, 1],
                                     [1, -1, -1],
                                     [-1, 1, -1],], 
                                    dtype=torch.float).unsqueeze(0)

  colors_tensor = torch.as_tensor([[255, 0, 0],
                                  [0, 255, 0],
                                  [0, 0, 255],
                                  [255, 0, 255],], 
                                  dtype=torch.int).unsqueeze(0)
  faces_tensor = torch.as_tensor([[0, 2, 3],
                                  [0, 3, 1],
                                  [0, 1, 2],
                                  [1, 3, 2],], 
                                 dtype=torch.int).unsqueeze(0)

  writer.add_mesh('my_mesh', vertices=vertices_tensor, colors=colors_tensor, faces=faces_tensor)

#----------------------------------------------------
# Hyper Parameters
#----------------------------------------------------
def add_hyper(writer):
  with writer as w:
    for i in range(5):
      w.add_hparams({'lr': 0.1*i, 'bsize': i},
                    {'hparam/accuracy': 10*i, 'hparam/loss': 10*i})

#----------------------------------------------------
# Scalars
#----------------------------------------------------
def add_scalars(writer):
  r = 5
  for n_iter in range(100):
    # Scalar values tagged in different hierarchies
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

    # Scalar values plotting a Straight Line
    writer.add_scalar('Plot/Line', n_iter * 2, n_iter)

def add_temp(writer):
  for n_iter in range(100):

    # Multiple scalar values for sin, cos, tan curves
    writer.add_scalars('Plot/run_14h', {'xsinx':n_iter*np.sin(n_iter/r),
                                      'xcosx':n_iter*np.cos(n_iter/r),
                                      'tanx': np.tan(n_iter/r)}, n_iter)

#----------------------------------------------------
# Image and Graph
#----------------------------------------------------
def add_image_graph(writer):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
  trainset = datasets.MNIST('mnist_train', train=True, download=True, transform=transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
  model = torchvision.models.resnet50(False)
  # Have ResNet model take in grayscale rather than RGB
  model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
  images, labels = next(iter(trainloader))

  grid = torchvision.utils.make_grid(images)
  writer.add_image('images', grid, 0)
  writer.add_graph(model, images)

def run_example(run_id):
  run_path = Path.cwd() / LOG_DIR / run_id 
  # Writer will output to ./runs/ directory by default
  writer = SummaryWriter(run_path)

  add_image_graph(writer)
  add_scalars(writer)
  add_hist(writer)
  #add_hyper(writer)
  #add_mesh(writer)

  writer.close()

run_example('run-1')
run_example('run-2')

### Temp

In [ ]:
#!rm -R tbtry
#!ls -lR tbtry

ls: cannot access 'tbtry': No such file or directory
